In [12]:
import pandas as pd
import regex as re

dataset = pd.read_csv("train_trimmed.csv")
# display(dataset)

/var/folders/hv/7qy2pzkn7w16gvn7th_1g2fm0000gn/T/ipykernel_14008/2469928685.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("train_trimmed.csv")


In [13]:
#Removing data with missing values equal to Nan 
dataset = dataset.dropna()

#drop the ID column
dataset = dataset.drop('ID', axis=1)

dataset['Annual_Income'] = dataset['Annual_Income'].str.replace('_', '')
dataset['Num_of_Delayed_Payment'] = dataset['Num_of_Delayed_Payment'].str.replace('_', '')
dataset['Num_of_Loan'] = dataset['Num_of_Loan'].str.replace('_', '')
dataset["Outstanding_Debt"] = dataset["Outstanding_Debt"].str.replace('_', '')
dataset["Monthly_Balance"] = dataset["Monthly_Balance"].str.replace('_', '')





#Remove the underscores from the age data entries 
dataset['Age'] = dataset['Age'].str.replace('_', '')
# display(dataset.head())
dataset = dataset.dropna()


In [14]:
dataset['Age'] = dataset['Age'].astype(int)
dataset = dataset[(dataset['Age'] >= 0) & (dataset['Age'] < 100)]

#display the age range 
display(dataset['Age'].min())
display(dataset['Age'].max())

14

95

In [15]:
#Iterating and converting the credit history to numerical values  
for i in dataset['Credit_History_Age']:
    years = (re.findall(r'(\d+) Years', i))    
    months = (re.findall(r'(\d+) Months', i))
    total_months = int(years[0]) * 12 + int(months[0])
    dataset['Credit_History_Age'] = dataset['Credit_History_Age'].replace(i, total_months)

display(dataset)

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Num_of_Delayed_Payment,Num_Credit_Inquiries,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Total_EMI_per_month,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",7,4.0,809.98,26.822620,265,49.574949,High_spent_Small_value_payments,312.49408867943663,Good
6,23,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",8,4.0,809.98,22.537593,271,49.574949,Low_spent_Small_value_payments,244.5653167062043,Good
8,28,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,4,2.0,605.03,24.464031,319,18.816215,Low_spent_Small_value_payments,470.69062692529184,Standard
9,28,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,1,2.0,605.03,38.550848,320,18.816215,High_spent_Large_value_payments,484.5912142650067,Good
10,28,34847.84,3037.986667,2,1385,6,1,Credit-Builder Loan,-1,2.0,605.03,33.224951,321,18.816215,High_spent_Large_value_payments,466.46647639764313,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98296,38,41015.55,3152.962500,0,4,2510,1,Payday Loan,1,2.0,599.4,31.469081,299,25.671257,Low_spent_Small_value_payments,346.7150840133413,Standard
98298,38,41015.55,3152.962500,0,4,4,1,Payday Loan,1,2.0,599.4,30.760830,301,25.671257,Low_spent_Small_value_payments,178.32714237042927,Standard
98299,38,41015.55,3152.962500,0,4,4,1,Payday Loan,1,2.0,599.4,31.936092,302,25.671257,Low_spent_Small_value_payments,192.0835536826656,Standard
98300,38,41015.55,3152.962500,1194,4,4,1,Payday Loan,1,2.0,599.4,26.268474,303,25.671257,High_spent_Large_value_payments,453.5677749786737,Standard


In [16]:
#Need to convert columns with numerical values and assert that they are all numerical
dataset.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())

Age                          True
Annual_Income                True
Monthly_Inhand_Salary        True
Num_Bank_Accounts            True
Num_Credit_Card              True
Interest_Rate                True
Num_of_Loan                  True
Type_of_Loan                False
Num_of_Delayed_Payment       True
Num_Credit_Inquiries         True
Outstanding_Debt             True
Credit_Utilization_Ratio     True
Credit_History_Age           True
Total_EMI_per_month          True
Payment_Behaviour           False
Monthly_Balance              True
Credit_Score                False
dtype: bool

In [22]:
#SHOULD PROBABLY PERFORM OUTLIER DETECTION FOR THESE instead of hard coding the range

def remove_outliers(df, column, multiplier):
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    # Calculate IQR (Interquartile Range)
    IQR = Q3 - Q1
    
    # Define the lower and upper bounds for outlier detection
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    # Filter outliers and return the DataFrame without outliers
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]


Columns = ['Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card','Interest_Rate', 'Num_of_Loan']
multiplier = 1.5  # Adjust strictness by changing the multipliers

dataset['Annual_Income'] = dataset["Annual_Income"].astype(float)
dataset["Monthly_Balance"] = dataset["Monthly_Balance"].astype(float)
dataset["Monthly_Inhand_Salary"] = dataset["Monthly_Inhand_Salary"].astype(float)
dataset["Num_Bank_Accounts"] = dataset["Num_Bank_Accounts"].astype(int)
dataset["Num_Credit_Card"] = dataset["Num_Credit_Card"].astype(int)
dataset["Interest_Rate"] = dataset["Interest_Rate"].astype(float)
dataset["Num_of_Loan"] = dataset["Num_of_Loan"].astype(int)
dataset["Num_of_Delayed_Payment"] = dataset["Num_of_Delayed_Payment"].astype(int)
dataset["Outstanding_Debt"] = dataset["Outstanding_Debt"].astype(float)

# dataset = dataset[dataset['Num_Bank_Accounts'] <= 13]
# dataset = dataset[dataset['Num_Credit_Card'] <= 12]
# dataset = dataset[dataset['Interest_Rate'] <= 19]
# dataset = dataset[dataset['Num_of_Loan'] <= 8]
dataset = dataset[dataset['Num_of_Delayed_Payment'] <= 23]
dataset = dataset[dataset['Num_Credit_Inquiries'] <= 15]
# dataset = dataset[dataset['Num_of_Delayed_Payment'] <= 18]

df_filtered = remove_outliers(dataset, "Annual_Income", multiplier=multiplier)


for column in Columns:
    df_filtered = remove_outliers(df_filtered, column, multiplier=multiplier)
    
df_filtered

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Num_of_Delayed_Payment,Num_Credit_Inquiries,...,Monthly_Balance,Credit_Score,Auto Loan,Credit-Builder Loan,Personal Loan,Payday Loan,Student Loan,Morgage Loan,Home Equity Loan,Consolidation Loan
0,23,19114.12,1824.843333,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",7,4.0,...,312.494089,Good,1,1,1,0,0,0,1,0
6,23,19114.12,1824.843333,3,4,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",8,4.0,...,244.565317,Good,1,1,1,0,0,0,1,0
8,28,34847.84,3037.986667,2,4,6.0,1,Credit-Builder Loan,4,2.0,...,470.690627,Standard,0,1,0,0,0,0,0,0
9,28,34847.84,3037.986667,2,4,6.0,1,Credit-Builder Loan,1,2.0,...,484.591214,Good,0,1,0,0,0,0,0,0
12,28,34847.84,3037.986667,2,4,6.0,1,Credit-Builder Loan,1,2.0,...,444.867032,Good,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98285,32,28796.82,2378.735000,6,6,21.0,5,"Student Loan, Payday Loan, Payday Loan, Person...",23,8.0,...,336.252200,Standard,0,0,1,1,1,0,0,0
98286,32,28796.82,2378.735000,6,6,21.0,5,"Student Loan, Payday Loan, Payday Loan, Person...",23,8.0,...,325.893766,Poor,0,0,1,1,1,0,0,0
98298,38,41015.55,3152.962500,0,4,4.0,1,Payday Loan,1,2.0,...,178.327142,Standard,0,0,0,1,0,0,0,0
98299,38,41015.55,3152.962500,0,4,4.0,1,Payday Loan,1,2.0,...,192.083554,Standard,0,0,0,1,0,0,0,0


In [23]:
# Remove Rows with invalid data
columns = ["Annual_Income", "Num_of_Delayed_Payment", "Num_of_Loan", "Outstanding_Debt", "Monthly_Balance", "Monthly_Inhand_Salary", "Num_Bank_Accounts", "Num_Credit_Card", "Interest_Rate", "Num_Credit_Inquiries", "Outstanding_Debt", "Total_EMI_per_month", "Credit_History_Age", "Credit_Utilization_Ratio"]

for col in columns:
    print(col)
    df_filtered = df_filtered.drop(df_filtered[df_filtered[col] < 0].index)

Annual_Income
Num_of_Delayed_Payment
Num_of_Loan
Outstanding_Debt
Monthly_Balance
Monthly_Inhand_Salary
Num_Bank_Accounts
Num_Credit_Card
Interest_Rate
Num_Credit_Inquiries
Outstanding_Debt
Total_EMI_per_month
Credit_History_Age
Credit_Utilization_Ratio


In [24]:
# One hot encoding for Type of loan 
# Find all unique regex matches with 'loan'
loan_matches = list(set(re.findall(r'\w+ Loan', ' '.join(dataset['Type_of_Loan']))))
print(loan_matches)

# Create a new column for each loan type
dataset['Auto Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Auto Loan', x) else 0)
dataset['Credit-Builder Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Credit-Builder Loan', x) else 0)
dataset['Personal Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Personal Loan', x) else 0)
dataset['Payday Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Payday Loan', x) else 0)
dataset['Student Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Student Loan', x) else 0)
dataset['Morgage Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Morgage Loan', x) else 0)
dataset['Home Equity Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Home Equity Loan', x) else 0)
dataset['Consolidation Loan'] = dataset['Type_of_Loan'].apply(lambda x: 1 if re.search(r'Consolidation Loan', x) else 0)

# Drop the original column
df_filtered = df_filtered.drop('Type_of_Loan', axis=1)


['Payday Loan', 'Builder Loan', 'Personal Loan', 'Student Loan', 'Consolidation Loan', 'Equity Loan', 'Mortgage Loan', 'Auto Loan']


In [25]:
# Look at dataframe
display(df_filtered.describe())

,Age,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Num_of_Delayed_Payment,Num_Credit_Inquiries,Outstanding_Debt,...,Total_EMI_per_month,Monthly_Balance,Auto Loan,Credit-Builder Loan,Personal Loan,Payday Loan,Student Loan,Morgage Loan,Home Equity Loan,Consolidation Loan
count,46768.000000,46768.000000,46768.000000,46768.000000,46768.000000,46768.000000,46768.000000,46768.000000,46768.000000,46768.000000,...,46768.000000,46768.000000,46768.000000,46768.000000,46768.000000,46768.000000,46768.000000,46768.0,46768.000000,46768.000000
mean,33.022387,45992.664373,3822.554840,5.479858,5.580461,14.958326,3.890801,13.354772,5.939317,1453.981120,...,1409.595016,374.599509,0.333754,0.350261,0.345728,0.355820,0.345621,0.0,0.345022,0.346177
std,10.707577,33031.457977,2747.738933,2.535303,2.054121,8.744418,2.164456,5.710288,3.760525,1154.259241,...,8250.592968,176.500307,0.471558,0.477057,0.475610,0.478766,0.475575,0.0,0.475381,0.475755
min,14.000000,7005.930000,303.645417,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.230000,...,4.462837,0.131136,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,24.000000,18760.600000,1576.815833,4.000000,4.000000,8.000000,2.000000,9.000000,3.000000,583.590000,...,39.767863,265.864693,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
50%,33.000000,35249.280000,2937.082500,6.000000,5.000000,14.000000,4.000000,14.000000,6.000000,1203.250000,...,75.319996,325.681327,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
75%,41.000000,66560.220000,5522.630000,7.000000,7.000000,20.000000,5.000000,18.000000,9.000000,2016.710000,...,160.769272,439.304513,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000
max,95.000000,147517.360000,11668.736667,11.000000,11.000000,34.000000,9.000000,23.000000,15.000000,4997.100000,...,82331.000000,1211.695329,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000


In [26]:
# Perform one hot encoding for Payment_Behavior
df_filtered = df_filtered[df_filtered['Payment_Behaviour'] != '!@9#%8']

payment_encoded = pd.get_dummies(df_filtered['Payment_Behaviour'])
payment_encoded = payment_encoded.astype(int)
df_filtered = pd.concat([df_filtered, payment_encoded], axis=1)

print(df_filtered.columns.tolist())

df_filtered.to_csv("train_processed.csv", sep=',', index=False, encoding='utf-8')




['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Num_of_Delayed_Payment', 'Num_Credit_Inquiries', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Total_EMI_per_month', 'Payment_Behaviour', 'Monthly_Balance', 'Credit_Score', 'Auto Loan', 'Credit-Builder Loan', 'Personal Loan', 'Payday Loan', 'Student Loan', 'Morgage Loan', 'Home Equity Loan', 'Consolidation Loan', 'High_spent_Large_value_payments', 'High_spent_Medium_value_payments', 'High_spent_Small_value_payments', 'Low_spent_Large_value_payments', 'Low_spent_Medium_value_payments', 'Low_spent_Small_value_payments']
